In [ ]:
import prehook

prehook.execute_prehook()

In [ ]:
import hook
import prehook
import database_handler
import lookups
import datetime

db_session = database_handler.create_connection()
# hook.create_etl_checkpoint(db_session)

In [ ]:
hook.return_lookup_items_as_dict(lookups.IncrementalField)

In [ ]:
import hook
import prehook
import database_handler
import lookups
import datetime

db_session = database_handler.create_connection()
etl_date = datetime.datetime(1900,1,1)
#SourceName is schema name
source_name = lookups.SourceName.DVD_RENTAL.value
#This will get you a list of tables inside this schema that are mentioned in SQLTablesToReplicate
tables = prehook.return_tables_by_schema(source_name)
#This will return 
incremental_date_dict = hook.return_lookup_items_as_dict(lookups.IncrementalField)
# for table in tables:
staging_query = f"""
        SELECT * FROM {source_name}.{tables[0]} WHERE {incremental_date_dict.get(tables[0])} >= '{etl_date}'
""" 
staging_df = database_handler.return_data_as_df(db_session= db_session, input_type= lookups.InputTypes.SQL, file_executor= staging_query)
dst_table = f"stg_{source_name}_{tables[0]}"
insert_stmt = database_handler.return_insert_into_sql_statement_from_df(staging_df, 'dw_reporting', dst_table)
for insert in insert_stmt:
        database_handler.execute_query(db_session=db_session, query= insert)

In [1]:
import misc_handler


In [4]:
df = misc_handler.return_match_df_from_web(66342,66343)

In [5]:
df

,match_id,date,home_team,away_team,home_score,away_score,home_possession_%,away_possession_%,home_shots_on_target,away_shots_on_target,...,home_corners,away_corners,home_offsides,away_offsides,home_yellow_cards,away_yellow_cards,home_red_cards,away_red_cards,home_fouls_conceded,away_fouls_conceded
0,66342,08/13/2021,Brentford,Arsenal,2,0,35.3,64.7,3.0,4.0,...,2.0,5.0,1.0,1.0,None,None,None,None,12.0,8.0


In [ ]:
df['Date'] = df['Date'].astype('datetime64[ns]')

In [ ]:
df.columns=df.columns.str.replace("_%","")

In [ ]:
df

In [ ]:
from prehook import first_time_web_scraping
first_time_web_scraping()